In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special

# Домашняя работа

Для задач Easy и Normal использовать данные Титаника


In [5]:
df = pd.read_csv("../hw_data/titanic/train.csv")
print(df.shape)
df.head(1)

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


## Easy

Оценить с помощью доверительных интервалов:
- выживаемость всех пассажиров (оценить среднее по isSurvived)
- выживаемость женщин
- выживаемость людей с каютами первого класса

Ответить на вопросы:
- какой вариант оценки доверительных интервалов нужно использовать в данной задаче и почему?
- какая из ошибок более критична для данной задачи - I или II рода?

### Ответы

Для оценки интервалов нужно использовать Т-распределение Стьюдента, т.к. истинная дисперсия неизвестна.

Если принять за гипотезу H0 человек выжил, то ошибкой первого рода будет признание живым погибшего. А второго рода - погибшим живого.
С точки зрения логики. первая ситуация более критична, т.к. живой в последствии сможет доказать факт, что он выжил.

In [6]:
def calc_interval(dfval):
    pas_s_mean = dfval["Survived"].mean()
    pas_s_std = dfval["Survived"].std()
    pas_count = dfval["PassengerId"].count()
    alpha = 0.05

    from scipy.stats import t

    t_interval = t.interval(1 - alpha, pas_count - 1, loc=pas_s_mean, scale=pas_s_std / np.sqrt(pas_count))
    print ("mean: "+str(pas_s_mean), t_interval, sep="\n")
    
calc_interval(df)

mean: 0.3838383838383838
(0.35184459896802966, 0.41583216870873796)


In [7]:
df_w = df[df["Sex"]=='female']
calc_interval(df_w)

mean: 0.7420382165605095
(0.6933807726141715, 0.7906956605068475)


In [8]:
df_1cl = df[df["Pclass"]==1]
calc_interval(df_1cl)

mean: 0.6296296296296297
(0.5647152190969751, 0.6945440401622842)


## Normal

- Проверить гипотезу о том, что женщины выживают чаще мужчин с уровнем значимости 0.05
- Проверить гипотезу о том, что дети выживают чаще, чем взрослые

H0 - женщины выживают чаще, H1 - мужчины выживают чаще

In [18]:
H0_mean = df[df["Sex"]=='female']["Survived"].mean()

result = scipy.stats.ttest_1samp( df[df["Sex"]=='male']["Survived"], H0_mean, alternative="greater")
result

Ttest_1sampResult(statistic=-33.91393230463904, pvalue=1.0)

pvalue = 1, следовательно верна гипотеза H0: женщины выживают чаще.

H0 - дети выживают чаще, H1 - взрослые выживают чаще

In [24]:
H0_mean = df[df["Age"]<18]["Survived"].mean()

result  = scipy.stats.ttest_1samp( df[df["Age"]>=18]["Survived"], H0_mean, alternative="greater") 
result

Ttest_1sampResult(statistic=-8.009176230313653, pvalue=0.999999999999997)

pvalue близко к 1, следовательно верна гипотеза H0: дети выживают чаще 

## Hard

Одной из важнейших тем в статистике является множественная проверка гипотез. 

Если нужно много раз проверять гипотезы, то вероятность совершить ошибку первого рода будет расти с каждой новой гипотезой.

[Прочитать подробнее можно здесь](https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D0%BF%D1%80%D0%B0%D0%B2%D0%BA%D0%B0_%D0%BD%D0%B0_%D0%BC%D0%BD%D0%BE%D0%B6%D0%B5%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D1%83%D1%8E_%D0%BF%D1%80%D0%BE%D0%B2%D0%B5%D1%80%D0%BA%D1%83_%D0%B3%D0%B8%D0%BF%D0%BE%D1%82%D0%B5%D0%B7)

Задача:

Мы отобрали 1000 человек, которые заявляют, что они экстрасенсы. Мы хотим проверить это. Для этого проводится следующий эксперимент. Перед каждым участником выкладывают 10 карт рубашкой вверх и просят угадать цвет карты (красный или черный). На уровне значимости 0.01 мы хотим быть уверены, является ли человек экстрасенсом.

Нужно смоделировать ситуацию двумя способами (предполагаем, что экстрасенсов не существует и решения участников представляют из себя биномиальное распределение):

- без множественной проверки гипотез, выявить количество экстрасенсов
- с множественной проверкой гипотез, используя поправку Бонферрони (правило FWER)

Сколько экстрасенсов получилось поймать в каждом из случаев?